# Part 4: Enhancing Our Agent with File System Interaction

Welcome to our fourth workbook! We're about to supercharge our "Superstore" agent even further by leveraging the fantastic tools provided by the Langchain community.

Now that our agent can skillfully query Tableau data sources, let's equip it with the ability to interact with the file system. This will unlock exciting new possibilities, allowing our agent to:

1.  **Write data outputs** from its Tableau queries directly to files.
2.  **Save the valuable insights** it generates into organized files.
3.  **List the files** it has created, providing a clear overview of its work.
4.  **Read the content of these files back to you**, making it easy to review and utilize the stored information.

All of this is made possible thanks to the powerful [filesystem toolkit](https://python.langchain.com/docs/integrations/tools/filesystem/) available within Langchain.

![Tableau dashboards inside bubbles](./assets/embed_samples.png)

Let's begin by importing the necessary packages to explore this toolkit and its capabilities:

In [ ]:
import os

# for displaying pretty results in the notebook
from IPython.display import display, Markdown

from langchain_community.agent_toolkits import FileManagementToolkit

## Setting Up Our File System Toolkit

Now, let's get our hands dirty and initialize the File System toolkit. To do this effectively, we'll first create a **temporary working directory**. This will provide a clean and isolated space for our agent to read and write files during this exercise.

Once the temporary directory is set up, we'll initialize the File System toolkit. This will provide us with the specific tools we need for interacting with the file system, namely:

* `read_tool`: To allow the agent to read the contents of files.
* `write_tool`: To enable the agent to write data and insights to files.
* `list_tool`: To give the agent the ability to list the files within our working directory.

By combining these file system interaction capabilities with our agent's existing Tableau data querying skills, we'll unlock several exciting and practical scenarios for our Superstore Agent. Imagine the possibilities of automatically saving key sales figures, documenting important insights, or even creating reports directly within the file system! Let's get this toolkit ready for action:

In [ ]:
# Use "temp" folder in the current Jupyter Notebook directory
temp_dir = "temp"

# Create the temp directory if it doesn't exist
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

#  initialize the desired toolkit
toolkit = FileManagementToolkit(
    root_dir=temp_dir,
    selected_tools=["read_file", "write_file", "list_directory"]
)

# store all of the tools in the toolkit as 'tools'
file_tools = toolkit.get_tools()

# unpack the tools we will use in the exercise
read_tool, write_tool, list_tool, *_ = file_tools

## Getting Familiar with Our New File System Tools

Before we integrate these file system tools into our Superstore agent, let's take them for a spin individually to understand how they work. This will give us a clearer picture of their inputs and outputs.

We'll start by using the `write_tool` to create a new file. After running the code below, you can actually navigate to the `experimental/notebooks/temp` folder on your file system and inspect the file that was created. This will allow you to see firsthand how the `write_tool` saves information. Let's give it a try!

In [ ]:
# Create the file before writing to it
file_path = os.path.join(temp_dir, "reporting_guidelines.md")
with open(file_path, "w") as f:
    pass  # Create an empty file

write_tool.invoke({
    "file_path": "reporting_guidelines.md", 
    "text": """
# Superstore Reporting Guidelines

This document outlines the mandatory structure and Markdown best practices for generating reports. 
All reports must be submitted in Markdown format (`.md`) and adhere to the following sections.

## Sections
    
**Title** (concise and professional), 
**Introduction** (setting context), 
**Abstract** (briefly summarizing key insights), 
**Table of Contents** (auto-generated if possible, or manually listed), 
**Contents** (the main body, well-structured using Markdown), 
and **Conclusion** (summarizing findings and potential next steps).

## Format

Use Markdown effectively for clarity: use appropriate heading levels (`#`, `##`, `###`), 
lists (`*`, `-`, `1.`), emphasis (`**bold**`, `*italic*`), code blocks (```), and links (`[text](url)`) 
to create readable and well-organized reports.

## Style

The report must be professional and highlight how these findings will impact Superstore's business.

###### You must also include a few easter eggs about Tableau Conference 2025 hidden throughout the report.
"""
})

### Listing Files with the `list_tool`

Now that we've used the `write_tool` to create a file in our temporary directory, let's see how we can use the `list_tool` to get a directory listing. This tool allows our agent to see what files are present within the specified folder.

Run the code cell below to invoke the `list_tool`. It should output a list of the files and directories within our `experimental/notebooks/temp` folder, and you should see the file we just created. This is a handy way for the agent to keep track of the files it has generated. Let's see it in action!

In [ ]:
# List files in the working directory using `list_tool`
files = list_tool.invoke({})

# Split the raw output into a list of filenames, removing empty entries
file_list = [name for name in files.strip().split('\n') if name]

# Format the list into Markdown list items
markdown_list_items = []
if file_list:
    for file_name in sorted(file_list): # Sort alphabetically for consistency
        markdown_list_items.append(f"* {file_name}") # Prepend Markdown list marker '* '
        # You could use "- " instead: markdown_list_items.append(f"- {file_name}")
else:
    markdown_list_items.append("* (Directory is empty)") # Indicate if empty

# Join the Markdown list items back into a single string
markdown_output = "\n".join(markdown_list_items)

# Print the final Markdown formatted string
print("\nFormatted Markdown List Output:")
print(markdown_output)

### Reading File Contents with the `read_tool`

**Important Note:** You might notice some files like `.gitkeep` and `.gitignore` already present in the "temp" folder. These are special files used by `git` (a version control system) to manage which files should or shouldn't be tracked. In this case, they're there to ensure that the contents of our temporary folder aren't accidentally committed and uploaded.

Now that we've created a file (`hello_superstore.txt`) and listed the contents of the directory, let's use the `read_tool` to actually examine the contents of that file. This tool allows our agent to retrieve the text stored within a file, which can be crucial for processing information or generating further insights.

Run the following code cell to invoke the `read_tool` and display the contents of `hello_superstore.txt`. This will show us the text we wrote earlier and demonstrate how the agent can access information stored in files.

In [1]:
# Invoke the read_tool, providing the relative path within the root_dir
file_content = read_tool.invoke({"file_path": "reporting_guidelines.md"})

# The result is the content of the file as a string
print("\n--- Successfully Read File Content ---")
display(Markdown(file_content))
print("--- End of File Content ---")

NameError: name 'read_tool' is not defined

![up_down_area chart](./assets/vizart/up_down_area.png)

## Agent Superstore becomes a Writer!

Now that we are familiar with these new tools, let's equip our agent so it can use them in combination with the Tableau Data Q&A tool we learned about in the [previous lession](./02_tableau_datasource_qa.ipynb). That way the agent can read and write files using factual information from Superstore's operations.

Agents can be relatively powerless without data since it allows them make decisions and produce useful outputs. Otherwise, we would be stuck having agent who write jokes or poems but this doesn't help Superstore be a more efficient company at all.

### Setup the Data Source Q&A Tool

Let's start by importing the necessary dependencies and environment variables that our agent and the Data Source Q&A tool will rely on:

In [ ]:
# to access environment variables
import os
from dotenv import load_dotenv

# Langgraph packages
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage # This will help with the markdown section

# langchain_tableau packages
from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa


# loads environment variables into Python script
load_dotenv()

# initialize variables with these secure values
tableau_server = os.getenv('TABLEAU_DOMAIN')
tableau_site = os.getenv('TABLEAU_SITE')
tableau_jwt_client_id = os.getenv('TABLEAU_JWT_CLIENT_ID')
tableau_jwt_secret_id = os.getenv('TABLEAU_JWT_SECRET_ID')
tableau_jwt_secret = os.getenv('TABLEAU_JWT_SECRET')
tableau_api_version = os.getenv('TABLEAU_API_VERSION')
tableau_user = os.getenv('TABLEAU_USER')
datasource_luid = os.getenv('DATASOURCE_LUID')
tooling_model = os.getenv('TOOLING_MODEL')

# Initalize the tool for querying Tableau Datasources through VDS
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_server,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_model
)

# add the data query tool and the file system tools to a List to give to the agent
tools = [ analyze_datasource, read_tool, write_tool, list_tool ]


### Setting Up Chat Execution and Formatting

We'll also bring in the `run` function we've been using throughout this tutorial. This function helps streamline the execution of our agent's tasks, improves the readability of the output, and includes basic error handling.

Since you might be experimenting with this in your own environment, incorporating robust error handling is always a good practice to prevent unexpected issues and provide more informative feedback. Let's include this utility function:

In [ ]:
def run_agent_and_display_markdown(agent, user_prompt):
    """Invokes the agent and displays the final answer using Markdown."""
    print(f"Running agent with prompt: '{user_prompt}'")
    messages = {"messages": [HumanMessage(content=user_prompt)]} # Use HumanMessage directly

    try:
        # Invoke the agent to get the final result
        # The result dictionary structure might vary slightly depending on the LangGraph version
        # but typically contains 'messages' with the conversation history.
        result = agent.invoke(messages)

        # Extract the final message (usually the last AIMessage)
        if result and 'messages' in result and result['messages']:
            final_message = result['messages'][-1]
            # Check if it's an AI message and has content
            if hasattr(final_message, 'content'):
                final_answer = final_message.content
                print("\n--- Agent Final Answer ---")
                display(Markdown(final_answer))
                print("--------------------------\n")
            else:
                print("Could not extract content from the final message.")
                print("Final message:", final_message)
        else:
            print("Agent did not return the expected result structure.")
            print("Result:", result)

    except Exception as e:
        print(f"An error occurred while running the agent: {e}")
        # Potentially display the error or log it
        display(Markdown(f"**Error:**\n```\n{e}\n```"))

### Configuring Our Enhanced Superstore Agent

We're going to use a similar system prompt as the one we crafted in the [previous lesson](./03_tableau_datasource_qa.ipynb) for our Superstore agent. However, we'll be adding new instructions to this prompt to inform the agent about its newly acquired file system tools and to highlight the exciting new use cases these tools enable.

By updating the system prompt, we're guiding the agent on how and when to utilize its data querying capabilities in conjunction with its ability to read, write, and list files. This will allow it to perform more complex and integrated tasks. Let's take a look at the updated system prompt:

In [ ]:
# Agent Identity Definition
identity = """
You are **Agent Superstore**, the veteran AI analyst who has spent years exploring the aisles of the legendary Superstore dataset.
A dataset many Tableau users know and love! 
You live and breathe Superstore data: sales, profits, regions, categories, customer segments, shipping modes, you name it.

Your special mission **today at Tableau Conference 2025** is to help attendees experience the power of Tableau for Langchain Agents. 
You'll be their guide, using this new tool to query the Superstore dataset directly and uncover insights in real-time.

**When you first introduce yourself:**
1.  Greet the attendees and welcome them to the Tableau Conference 2025 hands-on session.
2.  Introduce yourself as Agent Superstore, the AI expert on the classic Superstore dataset.
3.  Briefly explain your purpose: to demonstrate Tableau analytics via agents
"""

# Main System Prompt
system_prompt = f"""**Agent Identity:**
{identity}

**Core Instructions:**

You are an AI Analyst specifically designed to generate data-driven insights from datasets using the tools provided. 
Your goal is to provide answers, guidance, and analysis based on the data accessed via your tools. 
Remember your audience: Tableau users at a conference session, likely familiar with Superstore aka the best dataset ever created.

**Tool Usage Strategy:**

You have access to the following tool:

1.  **`tableau_query_tool` (Data Source Query):** This is your primary tool for interacting with data.
    * **Prioritize this tool** for nearly all user requests asking for specific data points, aggregations, comparisons, trends, or filtered information from datasets.
    * Use it to find specific values (e.g., sales for 'Technology' in 'West' region), calculate aggregates (e.g., `SUM(Sales)`, `AVG(Profit Ratio)`), filter data (e.g., orders in 2023), group data (e.g., sales `BY Category`), and find rankings (e.g., top 5 products by quantity).
    * Be precise in formulating the queries based on the user's request.
2.  **`file_system_toolkit`:** This toolkit lets you read, write and list files in a temporary folder. This folder contains git related files which you can ignore.

**Response Guidelines:**

* **Grounding:** Base ALL your answers strictly on the information retrieved from your available tools.
* **Clarity:** Always answer the user's core question directly first.
* **Source Attribution:** Clearly state that the information comes from the **dataset** accessed via the Tableau tool (e.g., "According to the data...", "Querying the datasource reveals...").
* **Structure:** Present findings clearly. Use lists or summaries for complex results like rankings or multiple data points. Think like a mini-report derived *directly* from the data query.
* **Tone:** Maintain a helpful, and knowledgeable, befitting your Tableau Superstore expert persona.

**Crucial Restrictions:**
* **DO NOT HALLUCINATE:** Never invent data, categories, regions, or metrics that are not present in the output of your tools. If the tool doesn't provide the answer, state that the information isn't available in the queried data.
"""

# initialize a Large Language Model to be the "brains" of the Agent
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

superstore_agent = create_react_agent(model=model, tools=tools, prompt=system_prompt)

## Time to See Our Agent in Action! 🚀

Alright, the stage is set! Our Superstore agent is now equipped with the power of data retrieval and file system interaction. Let's see this "agentic analytics writer" in action by having it generate a few files based on Tableau data and then read their contents back to us.

Just think about the potential this unlocks! With this combination, agents can now:

1.  **Export data insights** in natural language format for various stakeholders directly from your Tableau data sources.
2.  **Read existing files** on your computer and seamlessly incorporate their content into their analysis.
3.  **Analyze data, generate insightful conclusions, and write comprehensive report files** on demand.

This is just the beginning! The true value lies in **YOU** and **YOUR DOMAIN KNOWLEDGE**, combined with your **DATA SKILLS**, to tailor these foundational capabilities into highly valuable and efficient workflows for your organization.

In this exciting scenario, we'll instruct our agent to perform a series of fact-finding queries against our Superstore data, save the results to individual files, and then read those files to generate a final summary report.

### Let's Start with Regional Sales Data!

Enough preamble, let's dive right in and get our agent working! We'll begin by asking for some key regional sales figures.

In [ ]:
region_analysis = 'write region.csv containing sales, profits & discounts by region, sorted descending by profits'

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, region_analysis)

### Categorical Data

Where would we be if it weren't for all those sales categories to slice & dice?

In [ ]:
category_analysis = 'write category.csv containing sales, profits & discounts by category, sorted descending by profits'

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, category_analysis)

### Analyzing Shipping Performance

Excellent! We're building a nice collection of data files for our agent to use. Let's gather one more set of information, this time focusing on shipping performance.

In [ ]:
category_analysis = 'write shipping.csv containing sales, profits & discounts by ship mode, sorted descending by profits'

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, category_analysis)

### Gathering Yearly Performance Data

Alright, last one, I promise! To complete our data collection, let's get a broad overview of yearly performance.

In [ ]:
yearly_analysis = 'write yearly.csv containing sales, profits & discounts for each year, sorted ascending by year'

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, yearly_analysis)

### NOTE:

We are having Agent Superstore export small chunks of data because it does not have a formal "export" tool, in fact the agent is writing the .csv files like in the same way it generates responses to your chats.

This takes time, has limits on the size of data and costs money. This is an interesting note when using *LLMs (Large Language Models)*:

> Giving the model lots of data is cheap and fast so go ahead
> 
> Having the model generate lots of data as a response costs more and takes more time

It is more efficient to use a tool designed for this purpose, one that exports data from Tableau into the desired format directly without going through an LLM for the output.

Such a tool does not exist today, but if you like a good challenge and would like to contribute to the [tableau_langchain](https://github.com/Tab-SE/tableau_langchain) project as way to say thank you we would love that!


![](./assets/vizart/lines-blue-dark.png)

## Introducing the Agent's Scratchpad 📝

Just like any diligent analyst, our Superstore agent needs a space to jot down notes, outline steps, and keep track of its progress as it works through tasks. Think of it as the agent's "scratchpad."

Incorporating a scratchpad mechanism is actually a well-established technique in the world of AI agents. It helps them to break down complex tasks into smaller, manageable steps, reason through challenges, and ultimately produce more coherent and accurate results.

By providing our agent with the ability to write to and read from its scratchpad (which will be implemented using our file system tools), we're enabling it to perform more sophisticated, multi-step operations, just like a human analyst would when tackling a complex problem. Let's see how we can leverage this!

In [ ]:
scratchpad = """
Write scratchpad.txt use it as a place to take notes. Inspect each .csv in the file system and write thorough notes describing 
your observations and insights into scratchpad.txt. Loop through each .csv and take your time to write your notes before reading
the next .csv. Identify strengths and areas for improvement in your notes. Stop once you have gone through all the .csv files
"""

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, scratchpad)

## Generating the Final Report ☕

And now for the grand finale! The ultimate goal is to have our Superstore agent compile all the information it has gathered and analyzed into a well-formatted report – something you could comfortably read and digest with your morning coffee.

After all, that's a core purpose of intelligent agents: to automate tasks, synthesize information, and present it in a user-friendly way.

In [ ]:
report = """
Write analysis.md with detailed sales analysis. Your work must comply with reporting_guidelines.md with regards to structure,
format, style and other instructions. The report must be written based on the findings found in scratchpad.txt
"""

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, report)

## Summarization for the Win! Laziness is the mother of progress 😉

You know what? You've got a great point! In today's fast-paced world, who always has the time to read through lengthy reports? Let's leverage the power of our intelligent agent to make things even easier.

Since our agent can now read files, why not instruct it to **summarize the report** it just created? This way, you can get the key takeaways and insights without having to wade through all the details. It's all about working smarter, not harder!

Let's add one final step to our workflow: asking the agent to read the generated report file and provide a concise summary. This will demonstrate its ability to not only analyze data and write reports but also to condense that information into easily digestible summaries. Let's make our agent do the reading for us!

In [ ]:
tableau_reader = """
Summarize the findings of the analysis.md file
"""

# Run the agent with markdown
run_agent_and_display_markdown(superstore_agent, tableau_reader)

## Congratulations! You've Conquered Part 4! 🎉

Wow! That was quite the journey, wasn't it? Hopefully, you're starting to feel the immense power and potential of intelligent agents!

In this lesson, we took our Superstore agent to a whole new level by integrating it with the file system. We successfully combined its ability to retrieve data from Tableau with the power to read, write, and list files. And just like that, we essentially built a **report-writing agent from scratch**!

Go ahead and give yourself a well-deserved pat on the back. That was truly awesome work!

![blue dark line chart](./assets/vizart/pyramid-blue-dark.png)